In [24]:
from plio.io import io_controlnetwork
from knoten.csm import create_csm
from scipy import sparse
import ale
import csmapi
import os
import numpy as np

In [2]:
os.environ["ISISROOT"] = "/usgs/pkgs/isis3.7.0/install"
os.environ["ISIS3DATA"] = "/usgs/cpkgs/isis3/data"
import pysis
from pysis import isis

In [3]:
df = io_controlnetwork.from_isis('data/hand_dense.net')

In [10]:
df

,id,pointType,pointChoosername,pointDatetime,pointEditLock,pointIgnore,pointJigsawRejected,referenceIndex,aprioriSurfPointSource,aprioriSurfPointSourceFile,...,measureDatetime,measureEditLock,measureIgnore,measureJigsawRejected,diameter,apriorisample,aprioriline,samplesigma,linesigma,measureLog
0,autoseed_001,2,autoseed,2019-12-06T10:19:12,False,False,False,0,0,,...,2019-12-06T10:19:12,False,False,False,0.0,201.356777,302.569131,0.0,0.0,[]
1,autoseed_001,2,autoseed,2019-12-06T10:19:12,False,False,False,0,0,,...,2019-12-06T10:19:12,False,False,False,0.0,754.094710,15168.130157,0.0,0.0,[]
2,autoseed_001,2,autoseed,2019-12-06T10:19:12,False,False,False,0,0,,...,2019-12-06T10:19:12,False,False,False,0.0,399.936010,13312.074608,0.0,0.0,[]
3,autoseed_002,2,autoseed,2019-12-06T10:19:12,False,False,False,0,0,,...,2019-12-06T10:19:12,False,False,False,0.0,1926.464206,96.383931,0.0,0.0,[]
4,autoseed_002,2,autoseed,2019-12-06T10:19:12,False,False,False,0,0,,...,2019-12-06T10:19:12,False,False,False,0.0,2100.233957,14938.826738,0.0,0.0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,hand_15,2,ladoramkershner,2019-12-11T16:05:47,False,False,False,0,0,,...,2019-12-11T16:05:47,False,False,False,0.0,2413.917268,22462.671510,0.0,0.0,[]
126,hand_15,2,ladoramkershner,2019-12-11T16:05:47,False,False,False,0,0,,...,2019-12-11T16:05:47,False,False,False,0.0,2445.984758,24263.903615,0.0,0.0,[]
127,hand_16,2,ladoramkershner,2019-12-12T08:09:54,False,False,False,1,0,,...,2019-12-12T08:09:54,False,False,False,0.0,3843.915024,24115.819511,0.0,0.0,[]
128,hand_16,2,ladoramkershner,2019-12-12T08:09:54,False,False,False,1,0,,...,2019-12-12T08:09:54,False,False,False,0.0,4009.638555,9314.698759,0.0,0.0,[]


In [5]:
isd_files = []
sensors = {}
for line in open('data/cubes.lis'):
    basename = os.path.splitext(os.path.basename(line.strip()))[0]
    isd = os.path.join('data',basename+'.json')
    isd_files.append(isd)
    with open(isd, 'w+') as f:
        f.write(ale.loads(line.strip(), formatter='usgscsm'))
        
    sn = isis.getsn(from_=line.strip()).strip().decode('utf-8')
    sensors[sn] = create_csm(isd)


In [22]:
def compute_ground_points(df, sensors):
    '''
    df: control network data frame
    sensors: dict mapping serial numbers to CSM sensor models

    returns: dict mapping point IDs to ground points
    '''
    ground_points = {}
    for point_id, group in df.groupby('id'):
        num_measures = len(group)
        design_mat = np.zeros((num_measures*3,3))
        rhs = np.zeros((num_measures*3))
        for i in range(num_measures):
            row = group.iloc[i]
            measure = csmapi.ImageCoord(row['line'], row['sample'])
            locus = sensors[row['serialnumber']].imageToRemoteImagingLocus(measure)
            a = np.array([locus.point.x, locus.point.y, locus.point.z])
            d = np.array([locus.direction.x, locus.direction.y, locus.direction.z])
            d = d / np.linalg.norm(d)
            design_mat[3*i:3*i+3] = np.identity(3) - np.matmul(np.reshape(d, (3,1)), np.reshape(d,(1,3)))
            rhs[3*i:3*i+3] = np.dot(a,d)*d - a
        normal_mat = np.dot(design_mat.T, design_mat)
        ground_points[point_id] = np.dot(np.linalg.inv(normal_mat), np.dot(design_mat.T, rhs))
    return ground_points

In [39]:
ground_points = compute_ground_points(df, sensors)
print(ground_points)

{'autoseed_001': array([-2919362.22040891,  1194116.85675187, -1238062.88731626]), 'autoseed_002': array([-2923283.47830042,  1184949.12526661, -1238132.20987237]), 'autoseed_003': array([-2901675.06021349,  1187206.16589136, -1284192.11677878]), 'autoseed_004': array([-2905856.72700703,  1178133.93329453, -1284377.91638495]), 'autoseed_005': array([-2910117.30551087,  1169096.59183613, -1284608.7921495 ]), 'autoseed_006': array([-2904005.22117153,  1174757.89632878, -1293938.6856367 ]), 'autoseed_007': array([-2919491.62378745,  1187777.14526929, -1243463.16309986]), 'autoseed_008': array([-2923653.45250965,  1178677.44330708, -1243643.8688257 ]), 'autoseed_009': array([-2912871.00860269,  1193982.77930179, -1252696.22240812]), 'autoseed_010': array([-2916608.39587758,  1184743.29863285, -1252682.53773229]), 'autoseed_011': array([-2920545.24078439,  1175569.17362061, -1252769.79203113]), 'autoseed_012': array([-2910051.37736736,  1190817.35711992, -1261916.35672947]), 'autoseed_013':

In [31]:
def computeSensorPartials(sensors, control_network):
    cn_sensors = [sensors[sn] for sn in control_network['serialnumber']]
    ground_points = [csmapi.EcefCoord(x, y, z) for 
                     x, y, z in zip(control_network['adjustedX'],
                                    control_network['adjustedY'],
                                    control_network['adjustedZ'])]
    
    partials = [[sensor.computeAllSensorPartials(gp)] for 
                sensor, gp in zip(cn_sensors, ground_points)]
    return partials
        
def computeGroundPartials(sensors, control_network):
    cn_sensors = [sensors[sn] for sn in control_network['serialnumber']]
    ground_points = [csmapi.EcefCoord(x, y, z) for 
                    x, y, z in zip(control_network['adjustedX'],
                                    control_network['adjustedY'],
                                    control_network['adjustedZ'])]
    
    partials = [sensor.computeGroundPartials(gp) for sensor, gp in zip(cn_sensors, ground_points)]
    return partials

In [53]:
sensor_partials = computeSensorPartials(sensors, df)
print(len(sensor_partials))


ground_partials = computeGroundPartials(sensors, df)
print(len(ground_partials))


<class 'SwigPyObject'>
130
130


In [ ]:
def computeSensorPartials(sensors, control_network):
    cn_sensors = [sensors[sn] for sn in control_network['serialnumber']]
    ground_points = [csmapi.EcefCoord(x, y, z) for 
                     x, y, z in zip(control_network['adjustedX'],
                                    control_network['adjustedY'],
                                    control_network['adjustedZ'])]
    

    # Nested list comprehension evaluates left to right.
    #  For each sensor / ground point pair, compute the sensor partials for each
    #  parameters in that sensor.
    # returns line/sample partial as tuple
    partials = [sensor.computeSensorPartials(p_idx, gp) for 
                sensor, gp in zip(cn_sensors, ground_points) for
                p_idx in range(sensor.getNumParameters())]
    return partials
        
def computeGroundPartials(sensors, control_network):
    cn_sensors = [sensors[sn] for sn in control_network['serialnumber']]
    # returns x/y/z/ line , x/y/z sample
    ground_points = [csmapi.EcefCoord(x, y, z) for 
                    x, y, z in zip(control_network['adjustedX'],
                                    control_network['adjustedY'],
                                    control_network['adjustedZ'])]
    
    partials = [sensor.computeGroundPartials(gp) for sensor, gp in zip(cn_sensors, ground_points)]
    return partials